<a href="https://colab.research.google.com/github/Nov05/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/complaints_failed_banks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by Nov05 on 2019-06-25

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Consumer Complaints

In [4]:
# CAUTION: this dataset is 719 MB in size.

# this doesn't work for this large file 
# (seems that it would work for file under 50 MB)
# url = "https://drive.google.com/uc?id=1giGHkApdv3ehRI7uw4LoLhGT6rN_uR0r"

# this works
# url = "https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD"
# df = pd.read_csv(url)

# this works
path = "/content/drive/My Drive/data/cfpd_consumer_complaints/Consumer_Complaints.csv"
df = pd.read_csv(path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# describe non-numeric columns
df.describe(exclude=np.number)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?
count,1315085,1315085,1079919,1315085,779909,389859,466544,1315085,1294526,1195259,179965,710315,1315085,1315085,1315078,1315085,768501
unique,2760,18,76,167,218,372832,10,5321,63,22769,3,4,6,2709,8,2,2
top,09/08/2017,Mortgage,Credit reporting,Incorrect information on your report,Information belongs to someone else,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",CA,300XX,Servicemember,Consent provided,Web,09/08/2017,Closed with explanation,Yes,No
freq,3553,280698,237652,144799,65707,993,327095,120371,180075,7600,87235,390181,972343,3387,1019300,1282414,620123


In [6]:
df['datereceived'] = pd.to_datetime(df['Date received'], 
                                    format="%m/%d/%Y")
df['datesent'] = pd.to_datetime(df['Date sent to company'],
                                format="%m/%d/%Y")
df[['datereceived', 'datesent']].sample(2)

,datereceived,datesent
626166,2016-12-15,2016-12-16
781416,2016-03-02,2016-03-02


In [7]:
# total row count = 1315204
len(df['Complaint ID'].unique())

1315085

#### Consolidate changes in products

https://files.consumerfinance.gov/f/documents/201704_cfpb_Summary_of_Product_and_Sub-product_Changes.pdf  

In [23]:
df_prd = (df.fillna('-')
   .groupby(by=['Product', 'Sub-product'], as_index=False)
   .agg({'Complaint ID': pd.Series.nunique})
   .sort_values(by=['Product', 'Complaint ID'], 
                ascending=(True, False))
)
print(df_prd.shape)

(99, 3)


In [0]:
prds = df_prd['Product'].unique()
subs = df_prd['Sub-product'].unique()

In [0]:
# consolidate changes in products - such a pain

def new_prd(row):
  prd, sub = row[0], row[1]
  new_prd = ""
  
  # Debt collection (no change to product name)
  if prd in [prds[7]]:
    new_prd = prds[7]
  # Credit reporting, credit repair services, or other 
  # personal consumer reports (previously “Credit reporting”
  # and sub-products of “Other financial service”)
  elif prd in [prds[5], prds[6]]:
    new_prd = prds[6]
  # Mortgage (no change to product name)
  elif prd in [prds[10]]:
    new_prd = prds[10]
  # Credit card or prepaid card (previously “Credit card” 
  # and “Prepaid card”)  
  elif prd in [prds[3], prds[14], prds[4]]:
    new_prd = prds[4]
  # Checking or savings account (previously Bank account 
  # or service)  
  elif prd in [prds[0], prds[1]]:
    new_prd = prds[1]
  # Vehicle loan or lease (previously sub-product of 
  # Consumer loan)  
  elif prd==prds[16] or (prd==prds[2] and sub==subs[8]):
    new_prd = prds[16]
  # Student loan (no change to product name)  
  elif prd in [prds[15]]:
    new_prd = prds[15]
  # Payday loan, title loan, or personal loan (previously 
  # “Consumer loan” and “Payday loan”)
  elif prd in [prds[13], prds[12]] or (prd==prds[2] 
                                       and sub!=subs[8]):
    new_prd = prds[13]
  # Money transfer, virtual currency, or money service 
  # (previously “Money transfers,” “Other financial 
  # service,” and “Prepaid card”)  
  elif prd in [prds[8], prds[17], prds[9], prds[11], 
               prds[14]]:
    new_prd = prds[8]
  
  return new_prd

In [26]:
# consolidate products for the whole dataset
df['new'] = df.iloc[:,1:3].apply(new_prd, axis=1)
# verify consolidation
df[df['new']=='']

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,datereceived,datesent,comp,new


# Failed Banks  

In [0]:
# url="http://www.fdic.gov/bank/individual/failed/banklist_utf8.csv"
url = "https://drive.google.com/uc?id=16R-U0x0Cc3tt4UfzmtvTy3z7rjdW8AkJ"
df_banks = pd.read_csv(url, encoding='8859') # seriously? 8859?

In [0]:
# convert datetime 31-May-19
df_banks["closingdate"] = pd.to_datetime(df_banks["Closing Date"], 
                                         format="%d-%b-%y")
df_banks["updateddate"] = pd.to_datetime(df_banks["Updated Date"], 
                                         format="%d-%b-%y")

In [30]:
print(df_banks.shape)
df_banks.head(3)

(556, 9)


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date,closingdate,updateddate
0,The Enloe State Bank,Cooper,TX,10716,"Legend Bank, N. A.",31-May-19,18-Jun-19,2019-05-31,2019-06-18
1,Washington Federal Bank for Savings,Chicago,IL,30570,Royal Savings Bank,15-Dec-17,1-Feb-19,2017-12-15,2019-02-01
2,The Farmers and Merchants State Bank of Argonia,Argonia,KS,17719,Conway Bank,13-Oct-17,21-Feb-18,2017-10-13,2018-02-21


# join the datasets

In [33]:
df['comp'] = df['Company'].str.upper()
df['comp'].unique()

array(['NATIONWIDE RECOVERY GROUP LLC',
       'ONEMAIN FINANCIAL HOLDINGS, LLC.',
       'HCFS HEALTH CARE FINANCIAL SERVICES, INC.', ...,
       'JLM R.E. INVESTMENTS', 'VALLEY ESTATES ESCROW',
       'ASPEN HOME MORTGAGE GROUP'], dtype=object)

In [32]:
df_banks['bnk'] = df_banks['Bank Name'].str.upper()
df_banks['bnk'].unique()

array(['THE ENLOE STATE BANK', 'WASHINGTON FEDERAL BANK FOR SAVINGS',
       'THE FARMERS AND MERCHANTS STATE BANK OF ARGONIA',
       'FAYETTE COUNTY BANK',
       'GUARANTY BANK, (D/B/A BESTBANK IN GEORGIA & MICHIGAN)',
       'FIRST NBC BANK', 'PROFICIO BANK', 'SEAWAY BANK AND TRUST COMPANY',
       'HARVEST COMMUNITY BANK', 'ALLIED BANK',
       'THE WOODBURY BANKING COMPANY', 'FIRST CORNERSTONE BANK',
       'TRUST COMPANY BANK', 'NORTH MILWAUKEE STATE BANK',
       'HOMETOWN NATIONAL BANK', 'THE BANK OF GEORGIA', 'PREMIER BANK',
       'EDGEBROOK BANK', 'DORAL BANK',
       'CAPITOL CITY BANK & TRUST COMPANY', 'HIGHLAND COMMUNITY BANK',
       'FIRST NATIONAL BANK OF CRESTVIEW', 'NORTHERN STAR BANK',
       'FRONTIER BANK, FSB D/B/A EL PASEO BANK',
       'THE NATIONAL REPUBLIC BANK OF CHICAGO', 'NBRS FINANCIAL',
       'GREENCHOICE BANK, FSB', 'EASTSIDE COMMERCIAL BANK',
       'THE FREEDOM STATE BANK', 'VALLEY BANK',
       'SLAVIE FEDERAL SAVINGS BANK', 'COLUMBIA SAVINGS BANK'

In [34]:
df_banks['acq'] = df_banks['Acquiring Institution'].str.upper()
df_banks['acq'].unique()

array(['LEGEND BANK, N. A.', 'ROYAL SAVINGS BANK', 'CONWAY BANK',
       'UNITED FIDELITY BANK, FSB', 'FIRST-CITIZENS BANK & TRUST COMPANY',
       'WHITNEY BANK', 'CACHE VALLEY BANK', 'STATE BANK OF TEXAS',
       "TODAY'S BANK", 'UNITED BANK', 'THE BANK OF FAYETTE COUNTY',
       'TWIN CITY BANK', 'FIDELITY BANK', 'REPUBLIC BANK OF CHICAGO',
       'BANCO POPULAR DE PUERTO RICO', 'FIRST NBC BANK', 'BANKVISTA',
       'BANK OF SOUTHERN CALIFORNIA, N.A.', 'HOWARD BANK',
       'PROVIDENCE BANK, LLC', 'COMMUNITY & SOUTHERN BANK',
       'ALVA STATE BANK & TRUST COMPANY',
       'LANDMARK BANK, NATIONAL ASSOCIATION', 'GREAT SOUTHERN BANK',
       'BAY BANK, FSB', 'PALMETTO STATE BANK', 'FIRST CHOICE BANK',
       'WASHINGTONFIRST BANK', 'SUNWEST BANK', 'BANCFIRST',
       'SPIRIT OF TEXAS BANK, SSB', 'FIRST FEDERAL BANK OF FLORIDA',
       'PLAINSCAPITAL BANK', 'NO ACQUIRER',
       'FIRST FIDELITY BANK, NATIONAL ASSOCIATION', 'CB&S BANK, INC.',
       'NICOLET NATIONAL BANK', 'C1 BANK',

In [35]:
df1 = (pd.merge(df_banks, df, how='inner',
                left_on='bnk',
                right_on='comp'
               ))
print(df1.shape)
df1.head()

# didn't find anything for the failed banks

(0, 33)


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date,closingdate,updateddate,bnk,acq,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,datereceived,datesent,comp,new


In [36]:
df2 = (pd.merge(df_banks, df, how='inner',
                left_on='acq',
                right_on='comp'
               ))
print(df2.shape)
df2.head(2)

# there are some complaints about the Acquiring Institutions

(5666, 33)


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date,closingdate,updateddate,bnk,acq,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,datereceived,datesent,comp,new
0,Doral Bank,San Juan,PR,32102,Banco Popular de Puerto Rico,27-Feb-15,29-Jan-19,2015-02-27,2019-01-29,DORAL BANK,BANCO POPULAR DE PUERTO RICO,06/18/2019,"Money transfer, virtual currency, or money ser...",International money transfer,Fraud or scam,NaN,NaN,NaN,BANCO POPULAR DE PUERTO RICO,TX,NaN,Older American,NaN,Web,06/18/2019,Closed with explanation,Yes,NaN,3278572,2019-06-18,2019-06-18,BANCO POPULAR DE PUERTO RICO,"Money transfer, virtual currency, or money ser..."
1,Doral Bank,San Juan,PR,32102,Banco Popular de Puerto Rico,27-Feb-15,29-Jan-19,2015-02-27,2019-01-29,DORAL BANK,BANCO POPULAR DE PUERTO RICO,06/12/2019,Debt collection,Auto debt,False statements or representation,Indicated you were committing crime by not pay...,NaN,Company has responded to the consumer and the ...,BANCO POPULAR DE PUERTO RICO,FL,34741,NaN,Consent not provided,Web,06/12/2019,Closed with explanation,Yes,NaN,3272924,2019-06-12,2019-06-12,BANCO POPULAR DE PUERTO RICO,Debt collection


In [37]:
(df2.groupby(by='Acquiring Institution')
    .agg({"Complaint ID": pd.Series.nunique})
    .sort_values("Complaint ID", ascending=False)
)

,Complaint ID
Acquiring Institution,
Banco Popular de Puerto Rico,763
Synovus Bank,672
New York Community Bank,489
MidFirst Bank,307
IBERIABANK,226
Old National Bank,109
East West Bank,102
First Midwest Bank,79
City National Bank,68


In [38]:
(df2.groupby(by='new')
    .agg({"Complaint ID": pd.Series.nunique})
    .sort_values("Complaint ID", ascending=False)
)

,Complaint ID
new,
Mortgage,1059
Checking or savings account,993
Credit card or prepaid card,423
"Credit reporting, credit repair services, or other personal consumer reports",160
Debt collection,104
"Payday loan, title loan, or personal loan",55
"Money transfer, virtual currency, or money service",26
Vehicle loan or lease,26
Student loan,2


# Result: didn't find much information from the Failed Bank List.